In [67]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [69]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()

# Initialize optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([16, 6, 5, 5])
conv2.bias 	 torch.Size([16])
fc1.weight 	 torch.Size([120, 400])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([84, 120])
fc2.bias 	 torch.Size([84])
fc3.weight 	 torch.Size([10, 84])
fc3.bias 	 torch.Size([10])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}]


In [60]:
import torch
from torch.utils.data import Dataset

from json import JSONEncoder
import json

# Сохраняем словарь с весами в файл c именем fn (формат json) веса модели 
def save_model_json(fn, model):
    class EncodeTensor(JSONEncoder,Dataset):
        def default(self, obj):
            if isinstance(obj, torch.Tensor):
                return obj.cpu().detach().numpy().tolist()
            return super(NpEncoder, self).default(obj)

    with open(fn, 'w') as json_file:
        json.dump(model.state_dict(), json_file, cls=EncodeTensor)

# Загружаем из json файла с именем fn веса модели         
def load_model_json(fn):
    with open(fn, 'r') as json_file:
        state_dict = json.load(json_file)
    for layer in state_dict:
        state_dict[layer] = torch.Tensor(state_dict[layer])
    return state_dict

In [80]:
save_model_json('torch_w.json', model)

In [81]:
model.load_state_dict(load_model_json('torch_w.json')) 

<All keys matched successfully>

In [ ]:
model.to('cuda:0')

In [ ]:
t = torch.rand(100, 3, 32, 32)
t[1]

In [ ]:
model(t)